In [1]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook uses the mpilot library to build and run the EEMS model. The output is then joined to the original dataset using the `join_id` field using the Geopandas libary

**Input Data Layers:**
- Summarized Reporting Units (created in `4_Summarize.ipynb`)
    - Need both the csv and the shapefile


In [2]:
df = pd.read_csv("out_data/eems/A_RU_5.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Join_ID     18180 non-null  int64  
 1   apn         18179 non-null  object 
 2   Pr_Lg_Fr    18180 non-null  float64
 3   Pr_Fire_Ig  18180 non-null  float64
 4   Shape_Leng  18180 non-null  float64
 5   Shape_Area  18180 non-null  float64
 6   Mj_LU_Cat   18180 non-null  int64  
 7   WUI_Interm  18180 non-null  int64  
 8   WUI_Interf  18180 non-null  int64  
 9   geometry    18180 non-null  object 
 10  LU_Cat      18180 non-null  float64
dtypes: float64(5), int64(4), object(2)
memory usage: 1.5+ MB


**Create the mpilot command file**

In [45]:
def get_base_str(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fire_Ig = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fire_Ig",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUIy&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "LU_Cat",
    DataType = "Float",
    Metadata = [
        DisplayName: Landuse&nbsp;Category,
        ColorMap: RdYlBu_r
    ]
)

# Normalizations

Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Normalized&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fire_Ig = Normalize(
    InFieldName = Pr_Fire_Ig, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Normalized&nbsp;Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

# Operations

Hi_Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Fire_Ig], 
    Weights = [0.5, 0.5],
    Metadata = [
        DisplayName: High&nbsp;Fire&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)

Hi_Pri = WeightedSum(
    InFieldNames = [Hi_Fr_Rsk, WUI_Rsk, LU_Cat], 
    Weights = [0.333, 0.333, 0.334],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Area,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Hi_Pri, Hi_Fr_Rsk, WUI_Rsk, Nm_Lg_Fire, Nm_Fire_Ig, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fire_Ig, LU_Cat]
)
    """
    return base_str

In [46]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_5.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output.csv")
base_str = get_base_str(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

**Now we need to merge the outputted EEMS CSV with the rest of the data and create a shapefile for it. Also note we need to project it to UTM Zone 11N so that it visualizes in EEMS Online**

In [47]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output.csv"), pd.read_csv("out_data/eems/A_RU_5.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:26911")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/Final_EEMS_Output.shp")

**Now prep the Command File and the geodatabase for EEMS Online**

In [48]:
arcpy.ResetEnvironments()
gdb_name = "EEMS.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/Final_EEMS_Output.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")

In [49]:
f1 = open("EEMS_CMD_FILE.mpt", 'r')
f2 = open("EEMS_CMD_FILE_2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [50]:
arcpy.ClearWorkspaceCache_management()
del FeatureClass

**Now delete the EEMSWRITE command from the Command File and zip the geodatabase with the final EEMS feature class (and only that feature class)**